In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading the dataset
train = pd.read_csv("train.csv")

In [3]:
# creating group as per training_score

train_score = []


for score in train['avg_training_score']:
    if score <= 40 :
        train_score.append(0)
    elif score < 60 :
        train_score.append(1)
    elif score < 80 :
        train_score.append(2)    
    else:
        train_score.append(3)
        
# Create a column from the list
train['train_score'] = train_score

In [4]:
# missing value treatment

In [5]:
train['education'].replace(np.NaN, "Bachelor's", inplace = True)

In [7]:
table = train.pivot_table(values='previous_year_rating', index='department',columns='education',
                         aggfunc=np.median)
# Define function to return value of this pivot_table
def fpyr(x):
    return table.loc[x['department'],x['education']]
train['previous_year_rating'].fillna(train[train['previous_year_rating'].isnull()].apply(fpyr, axis=1), inplace=True)

In [8]:
train.isnull().sum()

employee_id             0
department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
is_promoted             0
train_score             0
dtype: int64

In [9]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train.gender = labelencoder.fit_transform(train.gender)
train.education = labelencoder.fit_transform(train.education)

In [10]:
# Independent Variable & Dependent Variable
X = train.loc[: , ['education','gender','previous_year_rating','KPIs_met >80%','awards_won?','train_score']]
Y = train.loc[: , ['is_promoted']]

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=7)

In [37]:
RF = RandomForestClassifier(n_estimators=3500,max_depth=3,oob_score=True,n_jobs = -1,random_state =50,
                            max_features = "sqrt", min_samples_leaf = 50)
RF.fit(X_train,Y_train.values.ravel())
scores = cross_val_score(RF, X_train, Y_train.values.ravel(), cv=10, scoring='f1')
Y_Pred = RF.predict(X_test)

In [38]:
p_s = precision_score(Y_test,Y_Pred)
r_s = recall_score(Y_test,Y_Pred)
cm = confusion_matrix(Y_test, Y_Pred)

In [39]:
print('Precision score is :', p_s)

Precision score is : 1.0


In [40]:
print('Recall score is :', r_s)

Recall score is : 0.021229050279329607


In [41]:
cm

array([[10067,     0],
       [  876,    19]], dtype=int64)

In [42]:
f_score_Rma = f1_score(Y_test.values.ravel(), Y_Pred, average='macro')
f_score_Rmi = f1_score(Y_test.values.ravel(), Y_Pred, average='micro')
f_score_Rwt = f1_score(Y_test.values.ravel(), Y_Pred, average='weighted')
f_score_Rno = f1_score(Y_test.values.ravel(), Y_Pred, average=None)

In [43]:
print("f1_score_macro for Random Forest is : ", f_score_Rma)
print("f1_score_micro for Random Forest is : ", f_score_Rmi)
print("f1_score_weighted for Random Forest is : ", f_score_Rwt)
print("f1_score_none for Random Forest is : ", f_score_Rno)

f1_score_macro for Random Forest is :  0.49994053055906484
f1_score_micro for Random Forest is :  0.9200875752599891
f1_score_weighted for Random Forest is :  0.8834585136399642
f1_score_none for Random Forest is :  [0.95830557 0.04157549]


# Predicting Test set

In [21]:
test = pd.read_csv("test.csv")

In [22]:
# creating group as per training_score

train_score = []


for score in test['avg_training_score']:
    if score <= 40 :
        train_score.append(0)
    elif score < 60 :
        train_score.append(1)
    elif score < 80 :
        train_score.append(2)    
    else:
        train_score.append(3)
        
# Create a column from the list
test['train_score'] = train_score

Missing Value Treatment

In [23]:
test['education'].replace(np.NaN, "Bachelor's", inplace = True)

In [25]:
table = test.pivot_table(values='previous_year_rating', index='department',columns='education',
                         aggfunc=np.median)
# Define function to return value of this pivot_table
def fpyr(x):
    return table.loc[x['department'],x['education']]
test['previous_year_rating'].fillna(test[test['previous_year_rating'].isnull()].apply(fpyr, axis=1), inplace=True)

In [26]:
# category to numeric conversion
labelencoder = LabelEncoder()
test.gender = labelencoder.fit_transform(test.gender)
test.education = labelencoder.fit_transform(test.education)

In [27]:
X_1 = test.loc[: , ['education','gender','previous_year_rating','KPIs_met >80%','awards_won?','train_score']]

In [28]:
RF.fit(X_train , Y_train.values.ravel())
is_promoted1 = RF.predict(X_1)

In [29]:
pd.DataFrame({"employee_id":test.employee_id , "is_promoted":is_promoted1}).set_index("employee_id").to_csv("submission3.csv")